<a href="https://colab.research.google.com/github/dipanjanS/improving-RAG-systems-dhs2024/blob/main/Demo_2_Solutions_for_Missed_Top_Ranked%2C_Not_in_Context%2C_Not_Extracted_%26_Incorrect_Specificity.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Solutions for Missed Top Ranked, Not in Context, Not Extracted & Incorrect Specificity



Here we will explore the following strategies

- Effect of Embedder Models
- Advanced Retrieval Strategies
- Chained Retrieval with Rerankers
- Context Compression Strategies

#### Install OpenAI, HuggingFace and LangChain dependencies

In [ ]:
!pip install langchain
!pip install langchain-openai
!pip install langchain-community
!pip install langchain-huggingface
!pip install langchain-chroma
!pip install rank_bm25

### Enter Open AI and HuggingFace API Tokens

In [ ]:
from getpass import getpass

OPENAI_KEY = getpass('Enter Open AI API Key: ')

Enter Open AI API Key: ··········


In [ ]:
from getpass import getpass

HUGGINGFACEHUB_API_TOKEN = getpass('Enter HuggingFace Auth Token Key: ')

Enter HuggingFace Auth Token Key: ··········


In [ ]:
import os

os.environ['OPENAI_API_KEY'] = OPENAI_KEY
os.environ['HUGGINGFACEHUB_API_TOKEN'] = HUGGINGFACEHUB_API_TOKEN

### Get Wikipedia Data

In [ ]:
!gdown 1oWBnoxBZ1Mpeond8XDUSO6J9oAjcRDyW

Downloading...
From (original): https://drive.google.com/uc?id=1oWBnoxBZ1Mpeond8XDUSO6J9oAjcRDyW
From (redirected): https://drive.google.com/uc?id=1oWBnoxBZ1Mpeond8XDUSO6J9oAjcRDyW&confirm=t&uuid=99c322c4-544a-4a41-83e2-103863d44341
To: /content/simplewiki-2020-11-01.jsonl.gz
100% 50.2M/50.2M [00:00<00:00, 57.3MB/s]


In [ ]:
import gzip
import json
from langchain.docstore.document import Document

wikipedia_filepath = 'simplewiki-2020-11-01.jsonl.gz'

docs = []
with gzip.open(wikipedia_filepath, 'rt', encoding='utf8') as fIn:
    for line in fIn:
        data = json.loads(line.strip())
        #Only add the first paragraph
        docs.append({
                        'metadata': {
                                        'title': data.get('title'),
                                        'article_id': data.get('id')
                        },
                        'data': data.get('paragraphs')[0] # restrict data to first 3 paragraphs to run later modules faster
        })

In [ ]:
docs = [doc for doc in docs for x in ['india']
              if x in doc['data'].lower().split()]
docs = [Document(page_content=doc['data'],
                 metadata=doc['metadata']) for doc in docs]
len(docs)

767

In [ ]:
docs[:3]

[Document(metadata={'title': 'Basil', 'article_id': '73985'}, page_content='Basil ("Ocimum basilicum") ( or ) is a plant of the Family Lamiaceae. It is also known as Sweet Basil or Tulsi. It is a tender low-growing herb that is grown as a perennial in warm, tropical climates. Basil is originally native to India and other tropical regions of Asia. It has been cultivated there for more than 5,000 years. It is prominently featured in many cuisines throughout the world. Some of them are Italian, Thai, Vietnamese and Laotian cuisines. It grows to between 30–60\xa0cm tall. It has light green, silky leaves 3–5\xa0cm long and 1–3\xa0cm broad. The leaves are opposite each other. The flowers are quite big. They are white in color and arranged as a spike.'),
 Document(metadata={'title': 'Roerich’s Pact', 'article_id': '259745'}, page_content='The Roerich Pact is a treaty on Protection of Artistic and Scientific Institutions and Historic Monuments, signed by the representatives of 21 states in the

# Exploring Embedding Models

In [ ]:
from langchain_huggingface.embeddings import HuggingFaceEmbeddings

# check out model details here: https://huggingface.co/sentence-transformers/multi-qa-MiniLM-L6-cos-v1
model_name = "sentence-transformers/multi-qa-MiniLM-L6-cos-v1"

bert_embedder = HuggingFaceEmbeddings(model_name=model_name)

In [ ]:
from langchain_chroma import Chroma

# create vector DB of docs and embeddings - takes < 30s on Colab
chroma_db1 = Chroma.from_documents(documents=docs,
                                  collection_name='wikipedia_db1',
                                  embedding=bert_embedder,
                                  collection_metadata={"hnsw:space": "cosine"})

retriever1 = chroma_db1.as_retriever(search_type="similarity",
                                     search_kwargs={"k": 2})

In [ ]:
query = "what is the capital of India?"
top_docs = retriever1.invoke(query)
top_docs

[Document(metadata={'article_id': '4062', 'title': 'Kolkata'}, page_content="Kolkata (spelled Calcutta before 1 January 2001) is the capital city of the Indian state of West Bengal. It is the second largest city in India after Mumbai. It is on the east bank of the River Hooghly. When it is called Calcutta, it includes the suburbs. This makes it the third largest city of India. This also makes it the world's 8th largest metropolitan area as defined by the United Nations. Kolkata served as the capital of India during the British Raj until 1911. Kolkata was once the center of industry and education. However, it has witnessed political violence and economic problems since 1954. Since 2000, Kolkata has grown due to economic growth. Like other metropolitan cities in India, Kolkata struggles with poverty, pollution and traffic congestion."),
 Document(metadata={'article_id': '352194', 'title': 'Gandhinagar'}, page_content='Gandhinagar is the capital city of Gujarat state in India. It is 23\xa

In [ ]:
query = "what is the old capital of India?"
top_docs = retriever1.invoke(query)
top_docs

[Document(metadata={'article_id': '5117', 'title': 'New Delhi'}, page_content='New Delhi () is the capital of India and a union territory of the megacity of Delhi. It has a very old history and is home to several monuments where the city is expensive to live in. In traditional Indian geography it falls under the North Indian zone. The city has an area of about 42.7\xa0km. New Delhi has a population of about 9.4 Million people.'),
 Document(metadata={'article_id': '352194', 'title': 'Gandhinagar'}, page_content='Gandhinagar is the capital city of Gujarat state in India. It is 23\xa0km from the city of Ahmedabad and 464\xa0km from Mumbai. In the year 1960, the Bombay state of India was divided into two states - Maharashtra and Gujarat. Bombay (now called Mumbai) became the capital city of Maharashtra. For Gujarat, new capital was needed. Gandhinagar was then made the capital of Gujarat.')]

In [ ]:
from langchain_huggingface.embeddings import HuggingFaceEmbeddings

# check out model details here: https://huggingface.co/mixedbread-ai/mxbai-embed-large-v1
model_name = "mixedbread-ai/mxbai-embed-large-v1"

mxbai_embedder = HuggingFaceEmbeddings(model_name=model_name)

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/171 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/113k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/677 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/1.24k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/695 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/297 [00:00<?, ?B/s]

In [ ]:
chroma_db2 = Chroma.from_documents(documents=docs,
                                  collection_name='wikipedia_db2',
                                  embedding=mxbai_embedder,
                                  collection_metadata={"hnsw:space": "cosine"})

retriever2 = chroma_db2.as_retriever(search_type="similarity",
                                     search_kwargs={"k": 2})

In [ ]:
query = "what is the capital of India?"
top_docs = retriever2.invoke(query)
top_docs

[Document(metadata={'article_id': '5117', 'title': 'New Delhi'}, page_content='New Delhi () is the capital of India and a union territory of the megacity of Delhi. It has a very old history and is home to several monuments where the city is expensive to live in. In traditional Indian geography it falls under the North Indian zone. The city has an area of about 42.7\xa0km. New Delhi has a population of about 9.4 Million people.'),
 Document(metadata={'article_id': '4062', 'title': 'Kolkata'}, page_content="Kolkata (spelled Calcutta before 1 January 2001) is the capital city of the Indian state of West Bengal. It is the second largest city in India after Mumbai. It is on the east bank of the River Hooghly. When it is called Calcutta, it includes the suburbs. This makes it the third largest city of India. This also makes it the world's 8th largest metropolitan area as defined by the United Nations. Kolkata served as the capital of India during the British Raj until 1911. Kolkata was once 

In [ ]:
query = "what is the old capital of India?"
top_docs = retriever2.invoke(query)
top_docs

[Document(metadata={'article_id': '5117', 'title': 'New Delhi'}, page_content='New Delhi () is the capital of India and a union territory of the megacity of Delhi. It has a very old history and is home to several monuments where the city is expensive to live in. In traditional Indian geography it falls under the North Indian zone. The city has an area of about 42.7\xa0km. New Delhi has a population of about 9.4 Million people.'),
 Document(metadata={'article_id': '4062', 'title': 'Kolkata'}, page_content="Kolkata (spelled Calcutta before 1 January 2001) is the capital city of the Indian state of West Bengal. It is the second largest city in India after Mumbai. It is on the east bank of the River Hooghly. When it is called Calcutta, it includes the suburbs. This makes it the third largest city of India. This also makes it the world's 8th largest metropolitan area as defined by the United Nations. Kolkata served as the capital of India during the British Raj until 1911. Kolkata was once 

In [ ]:
from langchain_openai import OpenAIEmbeddings

# details here: https://openai.com/blog/new-embedding-models-and-api-updates
openai_embed_model = OpenAIEmbeddings(model='text-embedding-3-small')

In [ ]:
chroma_db3 = Chroma.from_documents(documents=docs,
                                   collection_name='wikipedia_db3',
                                   embedding=openai_embed_model,
                                   collection_metadata={"hnsw:space": "cosine"})

retriever3 = chroma_db3.as_retriever(search_type="similarity",
                                     search_kwargs={"k": 2})

In [ ]:
query = "what is the capital of India?"
top_docs = retriever3.invoke(query)
top_docs

[Document(metadata={'article_id': '5117', 'title': 'New Delhi'}, page_content='New Delhi () is the capital of India and a union territory of the megacity of Delhi. It has a very old history and is home to several monuments where the city is expensive to live in. In traditional Indian geography it falls under the North Indian zone. The city has an area of about 42.7\xa0km. New Delhi has a population of about 9.4 Million people.'),
 Document(metadata={'article_id': '22215', 'title': 'States and union territories of India'}, page_content='The Republic of India is divided into twenty-eight States,and eight union territories including the National Capital Territory.')]

In [ ]:
query = "what is the old capital of India?"
top_docs = retriever3.invoke(query)
top_docs

[Document(metadata={'article_id': '5117', 'title': 'New Delhi'}, page_content='New Delhi () is the capital of India and a union territory of the megacity of Delhi. It has a very old history and is home to several monuments where the city is expensive to live in. In traditional Indian geography it falls under the North Indian zone. The city has an area of about 42.7\xa0km. New Delhi has a population of about 9.4 Million people.'),
 Document(metadata={'article_id': '352194', 'title': 'Gandhinagar'}, page_content='Gandhinagar is the capital city of Gujarat state in India. It is 23\xa0km from the city of Ahmedabad and 464\xa0km from Mumbai. In the year 1960, the Bombay state of India was divided into two states - Maharashtra and Gujarat. Bombay (now called Mumbai) became the capital city of Maharashtra. For Gujarat, new capital was needed. Gandhinagar was then made the capital of Gujarat.')]

# Exploring Advanced Retrieval & Reranking Strategies

### Multi Query Retrieval

Retrieval may produce different results with subtle changes in query wording, or if the embeddings do not capture the semantics of the data well. Prompt engineering / tuning is sometimes done to manually address these problems, but can be tedious.

The [`MultiQueryRetriever`](https://api.python.langchain.com/en/latest/retrievers/langchain.retrievers.multi_query.MultiQueryRetriever.html) automates the process of prompt tuning by using an LLM to generate multiple queries from different perspectives for a given user input query. For each query, it retrieves a set of relevant documents and takes the unique union across all queries to get a larger set of potentially relevant documents.

In [ ]:
from langchain_openai import ChatOpenAI

chatgpt = ChatOpenAI(model_name="gpt-4o", temperature=0)

In [ ]:
from langchain.retrievers.multi_query import MultiQueryRetriever
# Set logging for the queries
import logging

similarity_retriever3 = chroma_db3.as_retriever(search_type="similarity",
                                                search_kwargs={"k": 2})

mq_retriever = MultiQueryRetriever.from_llm(
    retriever=similarity_retriever3, llm=chatgpt,
    include_original=True
)

logging.basicConfig()
# so we can see what queries are generated by the LLM
logging.getLogger("langchain.retrievers.multi_query").setLevel(logging.INFO)

In [ ]:
query = "what is the capital of India?"
docs = mq_retriever.invoke(query)
docs

INFO:langchain.retrievers.multi_query:Generated queries: ['What city serves as the capital of India?', 'Which city is the administrative capital of India?', "Can you tell me the name of India's capital city?"]


[Document(metadata={'article_id': '5117', 'title': 'New Delhi'}, page_content='New Delhi () is the capital of India and a union territory of the megacity of Delhi. It has a very old history and is home to several monuments where the city is expensive to live in. In traditional Indian geography it falls under the North Indian zone. The city has an area of about 42.7\xa0km. New Delhi has a population of about 9.4 Million people.'),
 Document(metadata={'article_id': '4062', 'title': 'Kolkata'}, page_content="Kolkata (spelled Calcutta before 1 January 2001) is the capital city of the Indian state of West Bengal. It is the second largest city in India after Mumbai. It is on the east bank of the River Hooghly. When it is called Calcutta, it includes the suburbs. This makes it the third largest city of India. This also makes it the world's 8th largest metropolitan area as defined by the United Nations. Kolkata served as the capital of India during the British Raj until 1911. Kolkata was once 

In [ ]:
query = "what is the old capital of India?"
docs = mq_retriever.invoke(query)
docs

INFO:langchain.retrievers.multi_query:Generated queries: ['What was the capital of India before New Delhi?', 'Which city served as the capital of India in the past?', 'Can you tell me the former capital of India?']


[Document(metadata={'article_id': '5117', 'title': 'New Delhi'}, page_content='New Delhi () is the capital of India and a union territory of the megacity of Delhi. It has a very old history and is home to several monuments where the city is expensive to live in. In traditional Indian geography it falls under the North Indian zone. The city has an area of about 42.7\xa0km. New Delhi has a population of about 9.4 Million people.'),
 Document(metadata={'article_id': '352194', 'title': 'Gandhinagar'}, page_content='Gandhinagar is the capital city of Gujarat state in India. It is 23\xa0km from the city of Ahmedabad and 464\xa0km from Mumbai. In the year 1960, the Bombay state of India was divided into two states - Maharashtra and Gujarat. Bombay (now called Mumbai) became the capital city of Maharashtra. For Gujarat, new capital was needed. Gandhinagar was then made the capital of Gujarat.'),
 Document(metadata={'article_id': '4062', 'title': 'Kolkata'}, page_content="Kolkata (spelled Calcu

### Hybrid Search (BM25 + Semantic)

In [ ]:
laptops = [
    "The laptop model XPS13-9380 by Dell comes with an Intel Core i7-8565U processor, 16GB RAM, and 512GB SSD. It also features a 13.3-inch FHD display.",
    "Apple's MacBook Pro 16-inch model MVVJ2LL/A includes a 9th-generation Intel Core i9 processor, 16GB of RAM, and a 1TB SSD. It has a stunning Retina display with True Tone technology.",
    "The HP Spectre x360 15T-eb000 has an Intel Core i7-10750H processor, 16GB RAM, and a 512GB SSD. This model, 7DF22AV_1, also features a 4K UHD touch display.",
    "Lenovo's ThinkPad X1 Carbon Gen 8, part number 20U9005MUS, is equipped with an Intel Core i7-10510U, 16GB RAM, and a 1TB SSD. It is known for its lightweight design and durability.",
    "The ASUS ZenBook 14 UX434FL-DB77 features an Intel Core i7-8565U, 16GB RAM, and a 512GB SSD. This model also comes with a 14-inch FHD display and ScreenPad 2.0.",
    "Microsoft Surface Laptop 3 model VEF-00064 has an Intel Core i7-1065G7 processor, 16GB RAM, and a 256GB SSD. It is known for its sleek design and vibrant PixelSense display."
]

In [ ]:
laptop_db = Chroma.from_texts(laptops, collection_name='laptop_db',
                              embedding=openai_embed_model,
                              collection_metadata={"hnsw:space": "cosine"})

In [ ]:
from langchain.retrievers import BM25Retriever, EnsembleRetriever

similarity_retriever = laptop_db.as_retriever(search_type="similarity",
                                                search_kwargs={"k": 2})

bm25_retriever = BM25Retriever.from_texts(laptops)
bm25_retriever.k = 2

ensemble_retriever = EnsembleRetriever(
    retrievers=[bm25_retriever, similarity_retriever],
    weights=[0.7, 0.3]
)


In [ ]:
query = "laptops with 16GB RAM and processor i7-1065G7 intel"
docs = similarity_retriever.invoke(query)
docs

[Document(page_content="Lenovo's ThinkPad X1 Carbon Gen 8, part number 20U9005MUS, is equipped with an Intel Core i7-10510U, 16GB RAM, and a 1TB SSD. It is known for its lightweight design and durability."),
 Document(page_content='Microsoft Surface Laptop 3 model VEF-00064 has an Intel Core i7-1065G7 processor, 16GB RAM, and a 256GB SSD. It is known for its sleek design and vibrant PixelSense display.')]

In [ ]:
query = "laptops with 16GB RAM and processor i7-1065G7 intel"
docs = ensemble_retriever.invoke(query)
docs

[Document(page_content='Microsoft Surface Laptop 3 model VEF-00064 has an Intel Core i7-1065G7 processor, 16GB RAM, and a 256GB SSD. It is known for its sleek design and vibrant PixelSense display.'),
 Document(page_content='The ASUS ZenBook 14 UX434FL-DB77 features an Intel Core i7-8565U, 16GB RAM, and a 512GB SSD. This model also comes with a 14-inch FHD display and ScreenPad 2.0.'),
 Document(page_content="Lenovo's ThinkPad X1 Carbon Gen 8, part number 20U9005MUS, is equipped with an Intel Core i7-10510U, 16GB RAM, and a 1TB SSD. It is known for its lightweight design and durability.")]

### Chained Retrieval with Reranker

This strategy uses a chain of multiple retrievers sequentially to get to the most relevant documents. The following is the flow

Similarity Retrieval → Reranker Model Retrieval

In [ ]:
from langchain_community.cross_encoders import HuggingFaceCrossEncoder
from langchain.retrievers.document_compressors import CrossEncoderReranker
from langchain.retrievers import ContextualCompressionRetriever

# Retriever 1 - simple cosine distance based retriever
similarity_retriever = chroma_db3.as_retriever(search_type="similarity",
                                              search_kwargs={"k": 3})

# download an open-source reranker model - cross-encoder/qnli-electra-base
reranker = HuggingFaceCrossEncoder(model_name="cross-encoder/qnli-electra-base")
reranker_compressor = CrossEncoderReranker(model=reranker, top_n=2)
# Retriever 2 - Uses a Reranker model to rerank retrieval results from the previous retriever
final_retriever = ContextualCompressionRetriever(
    base_compressor=reranker_compressor,
    base_retriever=similarity_retriever
)

config.json:   0%|          | 0.00/771 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/438M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/268 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [ ]:
query = "what is the capital of India?"
docs = final_retriever.invoke(query)
docs

[Document(metadata={'article_id': '5117', 'title': 'New Delhi'}, page_content='New Delhi () is the capital of India and a union territory of the megacity of Delhi. It has a very old history and is home to several monuments where the city is expensive to live in. In traditional Indian geography it falls under the North Indian zone. The city has an area of about 42.7\xa0km. New Delhi has a population of about 9.4 Million people.'),
 Document(metadata={'article_id': '4062', 'title': 'Kolkata'}, page_content="Kolkata (spelled Calcutta before 1 January 2001) is the capital city of the Indian state of West Bengal. It is the second largest city in India after Mumbai. It is on the east bank of the River Hooghly. When it is called Calcutta, it includes the suburbs. This makes it the third largest city of India. This also makes it the world's 8th largest metropolitan area as defined by the United Nations. Kolkata served as the capital of India during the British Raj until 1911. Kolkata was once 

In [ ]:
query = "what is the old capital of India?"
docs = final_retriever.invoke(query)
docs

[Document(metadata={'article_id': '4062', 'title': 'Kolkata'}, page_content="Kolkata (spelled Calcutta before 1 January 2001) is the capital city of the Indian state of West Bengal. It is the second largest city in India after Mumbai. It is on the east bank of the River Hooghly. When it is called Calcutta, it includes the suburbs. This makes it the third largest city of India. This also makes it the world's 8th largest metropolitan area as defined by the United Nations. Kolkata served as the capital of India during the British Raj until 1911. Kolkata was once the center of industry and education. However, it has witnessed political violence and economic problems since 1954. Since 2000, Kolkata has grown due to economic growth. Like other metropolitan cities in India, Kolkata struggles with poverty, pollution and traffic congestion."),
 Document(metadata={'article_id': '352194', 'title': 'Gandhinagar'}, page_content='Gandhinagar is the capital city of Gujarat state in India. It is 23\xa

# Exploring Context Compression Strategies

### LLM Prompt-based Contextual Compression Retrieval

The context compression can happen in the form of:

- Remove parts of the content of retrieved documents which are not relevant to the query. This is done by extracting only relevant parts of the document to the given query

- Filter out documents which are not relevant to the given query but do not remove content from the document

Here we look at `LLMChainExtractor`, which will iterate over the initially returned documents and extract from each only the content that is relevant to the query. Totally irrelevant documents might also be dropped

In [ ]:
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import LLMChainExtractor
from langchain_openai import ChatOpenAI

chatgpt = ChatOpenAI(model_name="gpt-4o", temperature=0)

similarity_retriever = chroma_db3.as_retriever(search_type="similarity",
                                              search_kwargs={"k": 3})

# extracts from each document only the content that is relevant to the query
compressor = LLMChainExtractor.from_llm(llm=chatgpt)

# retrieves the documents similar to query and then applies the compressor
compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor, base_retriever=similarity_retriever
)

In [ ]:
query = "what is the capital of India?"
docs = compression_retriever.invoke(query)
docs

[Document(metadata={'article_id': '5117', 'title': 'New Delhi'}, page_content='New Delhi () is the capital of India and a union territory of the megacity of Delhi.')]

In [ ]:
query = "what is the old capital of India?"
docs = compression_retriever.invoke(query)
docs

[Document(metadata={'article_id': '4062', 'title': 'Kolkata'}, page_content='Kolkata served as the capital of India during the British Raj until 1911.')]

The `LLMChainFilter` is slightly simpler but more robust compressor that uses an LLM chain to decide which of the initially retrieved documents to filter out and which ones to return, without manipulating the document contents.

In [ ]:
from langchain.retrievers.document_compressors import LLMChainFilter

similarity_retriever = chroma_db3.as_retriever(search_type="similarity",
                                              search_kwargs={"k": 3})

#  decides which of the initially retrieved documents to filter out and which ones to return
_filter = LLMChainFilter.from_llm(llm=chatgpt)

# retrieves the documents similar to query and then applies the filter
compression_retriever = ContextualCompressionRetriever(
    base_compressor=_filter, base_retriever=similarity_retriever
)

In [ ]:
query = "what is the capital of India?"
docs = compression_retriever.invoke(query)
docs

[Document(metadata={'article_id': '5117', 'title': 'New Delhi'}, page_content='New Delhi () is the capital of India and a union territory of the megacity of Delhi. It has a very old history and is home to several monuments where the city is expensive to live in. In traditional Indian geography it falls under the North Indian zone. The city has an area of about 42.7\xa0km. New Delhi has a population of about 9.4 Million people.')]

In [ ]:
query = "what is the old capital of India?"
docs = compression_retriever.invoke(query)
docs

[Document(metadata={'article_id': '4062', 'title': 'Kolkata'}, page_content="Kolkata (spelled Calcutta before 1 January 2001) is the capital city of the Indian state of West Bengal. It is the second largest city in India after Mumbai. It is on the east bank of the River Hooghly. When it is called Calcutta, it includes the suburbs. This makes it the third largest city of India. This also makes it the world's 8th largest metropolitan area as defined by the United Nations. Kolkata served as the capital of India during the British Raj until 1911. Kolkata was once the center of industry and education. However, it has witnessed political violence and economic problems since 1954. Since 2000, Kolkata has grown due to economic growth. Like other metropolitan cities in India, Kolkata struggles with poverty, pollution and traffic congestion.")]